In [1]:
from spl.__main__ import SplManager
from spl.sync_manager import SyncManager
spl = SplManager(src="nxtp-onprem", dest="localhost", interactive=False)

In [2]:
src = spl._src
dest = spl._dest

sync_mg = SyncManager(spl, src, dest)

In [3]:
sync_mg.user_diff()

{
    'dictionary_item_removed': [root['analytics'], root['max']],
    'values_changed': {
        "root['admin']['last_successful_login']": {
            'new_value': '1640790841',
            'old_value': '1640775452'
        }
    },
    'iterable_item_removed': {
        "root['admin']['capabilities'][23]": 'edit_kvstore',
        "root['admin']['capabilities'][28]": 'edit_modinput_journald',
        "root['admin']['capabilities'][71]": 'ite_edit_objects',
        "root['admin']['capabilities'][123]": 'securegateway_read',
        "root['admin']['capabilities'][124]": 'securegateway_write'
    }
}

Users on 'nxtp-onprem' not present on '{self.dest._name}':

['analytics', 'max']

In [4]:
from deepdiff import DeepDiff

def user_diff(src, dest):
    return DeepDiff(
        {user.name: user.content for user in src._spl.users.list()},
        {user.name: user.content for user in dest._spl.users.list()},
        ignore_order=True,
    )

In [10]:
from typing import Tuple

class DiffConverter:
    """Logic to execute sync of diff from src to dest."""
    def __init__(self, diff_gen, sync_actions, src, dest):
        self._diff_gen = diff_gen
        self._src = src
        self._dest = dest
        self._sync_actions = sync_actions

    def _sync_missing(self, diff):
        """Sync completely missing entities (i.e. User, Index, ...)."""
        if "dictionary_item_removed" in diff:
            for item in diff["dictionary_item_removed"]:
                self._sync_actions["root"](self._src, self._dest, str(item).split("'")[1])

    @staticmethod
    def _sanitize_item_path(item) -> Tuple[str, str]:
        """Remove 'root' and unnecessary brackets aka convert to dot notation.
        
        Returns entity name and sanitized path as tuple.
        """
        split_path = item.replace("root", "").replace("][", ".").replace("[", "").replace("]", "").replace("'", "").split('.')
        return split_path[0], '.'.join(split_path[1:])

    def _sync_content(self, diff):
        """Synchronize all changes between src entity and dest entity."""
        if "values_changed" in diff:
            for item_path, value in diff["values_changed"].items():
                entity_name, sanitized_path = DiffConverter._sanitize_item_path(str(item_path))
                print(f"'{entity_name}': '{sanitized_path}' -> '{value['new_value']}'")
                try:
                    self._sync_actions[sanitized_path](
                        self._src, self._dest, entity_name, value["new_value"]
                    )
                except KeyError:
                    print(f"No action defined for '{sanitized_path}', thus skipping.")

        if "iterable_item_removed" in diff:
            for item_path, value in diff["iterable_item_removed"].items():
                entity_name, sanitized_path = DiffConverter._sanitize_item_path(str(item_path))
                sanitized_path = '.'.join(sanitized_path.split('.')[:-1])
                print(f"'{entity_name}': '{sanitized_path}' -> '{value}'")
                try:
                    self._sync_actions[sanitized_path](
                        self._src, self._dest, entity_name, value
                    )
                except KeyError:
                    print(f"No action defined for '{sanitized_path}', thus skipping.")

        # TODO add necessary logic for deletion of items on dest

    def sync_diff(self):
        diff = self._diff_gen(self._src, self._dest)
        self._sync_missing(diff)
        # if creation is properly implemented,
        # new diff will contain everything necessary to create for new user
        diff = self._diff_gen(self._src, self._dest)
        print(diff)
        self._sync_content(diff)



In [16]:
from spl.connection_adapter import ConnectionAdapter

def root_template(src: ConnectionAdapter, dest: ConnectionAdapter, entity_name: str, value: str):
    """TODO"""

def create_role(src: ConnectionAdapter, dest: ConnectionAdapter, _: str, value: str):
    """TODO"""
    dest._spl.roles.create(value)

def create_user(src, dest: ConnectionAdapter, entity_name):
    for user in src._spl.users.list():
        if user.name == entity_name:
            roles = []
            for role in user.role_entities:
                roles.append(role.name)
                create_role(src, dest, entity_name, role.name)
            dest._spl.users.create(entity_name, "UltraSecretPassword", roles)

def add_user_role(_, dest: ConnectionAdapter, entity_name, role_name):
    user = dest._spl.users.get(entity_name)
    user.role_entities.append(role_name)
    user.update()

In [17]:
dc = DiffConverter(user_diff, {
    "root": create_user,
    "capabilities": lambda *args, **kwargs: print(f"Simulate add of capa {args[-1]}"),
    "perm.read": lambda *args, **kwargs: print(f"Simulate add of capa {args[-1]}")
}, src, dest)
dc.sync_diff()

In [16]:
spl._src.indexes()

- _audit

- _internal

- _introspection

- _telemetry

- _thefishbucket

...

- main

- o365

- os_unix

- splunklogger

- summary